In [ ]:
# Verificar GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# Verificar Memoria
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
# Importaciones
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Verificar si hay GPU
print(f'GPU disponible: {torch.cuda.is_available()}')

In [ ]:
# Nombre del modelo
model_name = "codellama/CodeLlama-13b-Instruct-hf"

In [ ]:
# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Configurar el token de relleno

In [ ]:
# Cargar el modelo en media precisión (fp16)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16  # Reduce el uso de memoria
)

In [ ]:
# Prompt de prueba
prompt = """
# Write a Python Function called analyze_name. The parameter is `full_name`, which contains the user's full name as a string. The function should return a JSON response with the following information:
# 1. The total number of words in the full name.
# 2. The number of letters in each word.
# 3. A breakdown of the letters present in the full name and their frequencies.
"""

In [ ]:
# Preparar la entrada
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,  # Añade relleno si es necesario
    truncation=True,  # Trunca si el texto es demasiado largo
    max_length=700,  # Define una longitud máxima clara
    return_attention_mask=True  # Asegura que se incluya el attention_mask
).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Generar la respuesta con parámetros ajustados
outputs = model.generate(
    inputs.input_ids,
    max_length=5000,  # Incrementa el número de tokens para respuestas más completas
    temperature=0.7,  # Controla la creatividad
    do_sample=True,   # Habilita el muestreo
    top_k=50,         # Considera solo las 50 palabras más probables en cada paso
    top_p=0.95,        # Habilita la selección basada en probabilidad acumulada
    repetition_penalty=1.2,  # Penaliza repeticiones, valores > 1 reducen repeticiones
    eos_token_id=tokenizer.eos_token_id  # Termina la generación en el token <eos>
)

In [ ]:
# Decodificar y mostrar la respuesta
print(tokenizer.decode(outputs[0], skip_special_tokens=True))